In [ ]:
import pandas as pd
from environments.forex_trading import tgym

In [ ]:
file = "./data/split/GBPUSD/weekly/GBPUSD_2017_0.csv"
#file = "./data/split/GBPUSD/weekly/GBPUSD_20191208_W_1523.csvx"
# file = "./data/split/GBPUSD/weekly/GBPUSD_20190915_W_1401.csvx"
df = pd.read_csv(file)
t = tgym(df,env_config_file='./data/config/gdbusd-test-1.json' )

In [ ]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# if model: del model # remove to demonstrate saving and loading
model_name='./data/models/GBPUSD-week-20211023081447'
model = PPO.load(model_name)

start_time = time.time()
obs = t.reset()
t.current_step=0
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = t.step(action)
    t.render(mode='human')
print(f"--- running time: {(time.time() - start_time)}---")

In [ ]:
for i in t.transaction_history:
    if i["Status"] == 3:
        print(i)


In [ ]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments

start_time = time.time()
model_name = "./data/models/GBPUSD-week-" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
print(f'save model as {model_name}')
model = PPO("MlpPolicy", t, verbose=1)
model.learn(total_timesteps=250000)
model.save(model_name)
print(f"--- trainning time: {(time.time() - start_time)}---")
